<a href="https://colab.research.google.com/github/venkat2ram/Keras-and-Tensorflow/blob/master/ud_L0602_TransferLearning_practice_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import tensorflow.keras.utils
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [2]:
URL='https://www.tensorflow.org/datasets/catalog/tf_flowers'
(training_examples,validation_examples),info=tfds.load('tf_flowers',split=['train[0:80%]','train[80%:]'],as_supervised=True,with_info=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


In [3]:
num_examples=info.splits['train'].num_examples

In [4]:
num_classes=info.features['label'].num_classes

In [5]:
num_examples, num_classes

(3670, 5)

In [25]:
info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
})

In [14]:
IMAGE_RES=224
BATCH_SIZE=32

In [15]:
def format_image(image,label):
  image=tf.image.resize(image,size=(IMAGE_RES,IMAGE_RES))/255.0
  return image,label

train_batches=training_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
val_batches=validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [16]:
#model_url='https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

model_url='https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

feature_extractor=hub.KerasLayer(model_url,input_shape=(IMAGE_RES,IMAGE_RES,3))

In [17]:
feature_extractor.trainable=False

In [27]:
model=tf.keras.Sequential([feature_extractor,layers.Dense(5)])

In [28]:
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [31]:
model.fit(train_batches,epochs=6,batch_size=BATCH_SIZE,validation_data=val_batches)

Epoch 1/6
92/92 [==============================] - 8s 83ms/step - loss: 1.7829 - accuracy: 0.2572 - val_loss: 1.6094 - val_accuracy: 0.2452
Epoch 2/6
92/92 [==============================] - 6s 70ms/step - loss: 1.6094 - accuracy: 0.2677 - val_loss: 1.6094 - val_accuracy: 0.2452
Epoch 3/6
92/92 [==============================] - 7s 74ms/step - loss: 1.6094 - accuracy: 0.2677 - val_loss: 1.6094 - val_accuracy: 0.2452
Epoch 4/6
92/92 [==============================] - 7s 74ms/step - loss: 1.6094 - accuracy: 0.2677 - val_loss: 1.6094 - val_accuracy: 0.2452
Epoch 5/6
92/92 [==============================] - 7s 74ms/step - loss: 1.6094 - accuracy: 0.2677 - val_loss: 1.6094 - val_accuracy: 0.2452
Epoch 6/6
92/92 [==============================] - 7s 73ms/step - loss: 1.6094 - accuracy: 0.2677 - val_loss: 1.6094 - val_accuracy: 0.2452


In [ ]:
image_batch

In [ ]:
np.array(image)

In [ ]:
num_examples//4